# Recommendation System 

In [1]:
import numpy as np
import ast
import pandas as pd

<h3>Preprocessing of the datasets</h3><br>
Read the datasets 

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

We have two data files so join the data files and make it a single data frame

In [3]:
movies = movies.merge(credits, on='title')

In [4]:
movies.head(500)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,79000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 28, ""...",http://www.themysteriousisland.com/,72545,"[{""id"": 10364, ""name"": ""mission""}, {""id"": 4089...",en,Journey 2: The Mysterious Island,Sean Anderson partners with his mom's boyfrien...,40.723459,"[{""name"": ""New Line Cinema"", ""id"": 12}, {""name...",...,94.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Believe the Impossible. Discover the Incredible.,Journey 2: The Mysterious Island,5.8,1030,72545,"[{""cast_id"": 12, ""character"": ""Hank"", ""credit_...","[{""credit_id"": ""52fe4871c3a368484e0f87b9"", ""de..."
496,78000000,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...",NaN,109451,"[{""id"": 1436, ""name"": ""inventor""}, {""id"": 1063...",en,Cloudy with a Chance of Meatballs 2,After the disastrous food storm in the first f...,41.247402,"[

Required columns to create tags:
1. genres
2. movie_id (to fetch images)
3. keywords
4. title
5. overview
6. cast
7. crew
8. Tagline (we will convert overview string into list and store here)

In [5]:
 movies = movies[['movie_id','overview','tagline','genres','keywords','title','cast','crew']]

In [6]:
movies.head(1)

,movie_id,overview,tagline,genres,keywords,title,cast,crew
0,19995,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


Check for missing data if any

In [7]:
movies.isnull().sum()

movie_id      0
overview      3
tagline     844
genres        0
keywords      0
title         0
cast          0
crew          0
dtype: int64

In [8]:
#3 rows of overview are missing so we will remove those
movies.dropna(inplace=True)

In [9]:
# check for duplicate data
movies.duplicated().sum()

0

Format the data

In [10]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [11]:
def format(obj):
  Lis = []
  for i in ast.literal_eval(obj): #convert string into list
    Lis.append(i['name'])
  return Lis

In [12]:
movies['genres'].apply(format)

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4801                                          [Drama]
4802               [Science Fiction, Drama, Thriller]
4804                        [Action, Crime, Thriller]
4805                                [Comedy, Romance]
4807                                               []
Name: genres, Length: 3965, dtype: object

In [13]:
movies['genres'] = movies['genres'].apply(format)

In [14]:
movies['keywords'] = movies['keywords'].apply(format)

In [15]:
def formatCast(obj):
  Lis1 = []
  counter=0
  for i in ast.literal_eval(obj):
    if(counter<3):
      Lis1.append(i['name'])
      counter += 1
    else:
      break
  return Lis1

In [16]:
movies['cast'] = movies['cast'].apply(formatCast)

In [17]:
#fetch the names where job is Director
def fetch_director(obj):
  Lis = []
  for i in ast.literal_eval(obj):
    if i['job'] == 'Director':
      Lis.append(i['name'])
      break
  return Lis


In [18]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [19]:
movies.head()

,movie_id,overview,tagline,genres,keywords,title,cast,crew
0,19995,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...",Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,Following the death of District Attorney Harve...,The Legend Ends,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,"John Carter is a war-weary, former military ca...","Lost in our world, found in another.","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...",John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


In [20]:
#converting overview into list and storing it into tagline
movies['tagline'] = movies['overview'].apply(lambda x:x.split())

Remove the spaces bewtween the words because each word will become a separate tag for eg we have two names 'Sam Worthington' and 'Sam Mendes' so the sam tag can confuse are model

In [21]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [22]:
movies.head()

,movie_id,overview,tagline,genres,keywords,title,cast,crew
0,19995,"In the 22nd century, a paraplegic Marine is di...","[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...",Avatar,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,"Captain Barbossa, long believed to be dead, ha...","[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...",Pirates of the Caribbean: At World's End,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,A cryptic message from Bond’s past sends him o...,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...",Spectre,"[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,Following the death of District Attorney Harve...,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...",The Dark Knight Rises,"[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,"John Carter is a war-weary, former military ca...","[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...",John Carter,"[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


In [23]:
#Create a new tags column
movies['tags'] = movies['tagline'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [24]:
new_df = movies[['movie_id','title','tags','overview','cast']]

In [25]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\SONY\AppData\Local\Temp\ipykernel_8540\3089450492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


In [26]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

C:\Users\SONY\AppData\Local\Temp\ipykernel_8540\3214958533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


<strong>Final data frame of the project</strong>

In [27]:
new_df.head()

,movie_id,title,tags,overview,cast
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di...","In the 22nd century, a paraplegic Marine is di...","[SamWorthington, ZoeSaldana, SigourneyWeaver]"
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha...","Captain Barbossa, long believed to be dead, ha...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]"
2,206647,Spectre,a cryptic message from bond’s past sends him o...,A cryptic message from Bond’s past sends him o...,"[DanielCraig, ChristophWaltz, LéaSeydoux]"
3,49026,The Dark Knight Rises,following the death of district attorney harve...,Following the death of District Attorney Harve...,"[ChristianBale, MichaelCaine, GaryOldman]"
4,49529,John Carter,"john carter is a war-weary, former military ca...","John Carter is a war-weary, former military ca...","[TaylorKitsch, LynnCollins, SamanthaMorton]"


Stemming new data frame tags: the process of producing morphological variants of a root/base word. A stemming algorithm reduces the words “chocolates”, “chocolatey”, “choco” to the root word, “chocolate” and “retrieval”, “retrieved”, “retrieves” reduce to the stem “retrieve”.

In [28]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [29]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)

In [30]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\SONY\AppData\Local\Temp\ipykernel_8540\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


<h3>Vectorization</h3>

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [32]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [33]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [34]:
# Most frequent words
cv.get_feature_names()

C:\Users\SONY\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['000',
 '007',
 '10',
 '100',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '17th',
 '18',
 '1890',
 '18th',
 '19',
 '1930',
 '1940',
 '1950',
 '1950s',
 '1960',
 '1960s',
 '1970',
 '1970s',
 '1974',
 '1980',
 '1990',
 '19th',
 '19thcenturi',
 '20',
 '200',
 '20th',
 '21st',
 '24',
 '25',
 '30',
 '300',
 '3d',
 '40',
 '50',
 '500',
 '60',
 '70',
 'aaron',
 'aaroneckhart',
 'aarontaylor',
 'abandon',
 'abduct',
 'abigailbreslin',
 'abil',
 'abl',
 'aboard',
 'aborigin',
 'abov',
 'abus',
 'academ',
 'academi',
 'academy',
 'accept',
 'access',
 'accid',
 'accident',
 'accompani',
 'accomplish',
 'account',
 'accus',
 'ace',
 'achiev',
 'acquaint',
 'acquir',
 'act',
 'action',
 'actionhero',
 'activ',
 'activist',
 'activities',
 'actor',
 'actress',
 'actual',
 'ad',
 'adam',
 'adammckay',
 'adamsandl',
 'adamshankman',
 'adapt',
 'add',
 'addict',
 'adjust',
 'admir',
 'admit',
 'adolesc',
 'adopt',
 'ador',
 'adrienbrodi',
 'adrift',
 'adult',
 'adulteri',
 'adulthood',
 'advanc

In [35]:
from sklearn.metrics.pairwise import cosine_similarity

In [36]:
similarity = cosine_similarity(vectors) #2D array

In [37]:
#Similarity of first movie with all the other movies
list(enumerate(similarity[1]))

[(0, 0.08238525545716345),
 (1, 1.0),
 (2, 0.06063390625908323),
 (3, 0.03883678186903086),
 (4, 0.07669649888473704),
 (5, 0.11562432251572008),
 (6, 0.02111001654603745),
 (7, 0.1292719224987548),
 (8, 0.06160411036336974),
 (9, 0.10226199851298272),
 (10, 0.07504691899113898),
 (11, 0.4140043409440133),
 (12, 0.0942285694863091),
 (13, 0.1355815361366601),
 (14, 0.11163607017227663),
 (15, 0.08346223261119858),
 (16, 0.2533201985524494),
 (17, 0.059808916712167495),
 (18, 0.1097907187727647),
 (19, 0.061357199107789626),
 (20, 0.05797710356524483),
 (21, 0.10727978097782273),
 (22, 0.09553946329477228),
 (23, 0.14097096860865022),
 (24, 0.10969086361906959),
 (25, 0.10846522890932808),
 (26, 0.0850871259623034),
 (27, 0.06063390625908323),
 (28, 0.04624972900628803),
 (29, 0.11778571185788637),
 (30, 0.09723055853282464),
 (31, 0.0893088561378213),
 (32, 0.10289915108550529),
 (33, 0.03300491809922247),
 (34, 0.10502100630210075),
 (35, 0.07756315349925287),
 (36, 0.0563883874434600

In [38]:
movie_list = []
movie_list.append(list(enumerate(similarity[12]))[12])
print(movie_list)
movie_list = sorted(list(enumerate(similarity[0])), reverse = True, key = lambda x:x[1])[1:6]
print(movie_list)


[(12, 1.0)]
[(1170, 0.2755193843889919), (2227, 0.25692800972687846), (525, 0.2508726030021272), (494, 0.25016291953504993), (567, 0.24194822861802104)]


In [39]:
def suggest(movie):
    movieIndex = new_df[new_df['title']==movie].index[0] #gives index of the movie
    distances = similarity[movieIndex] # distance of that movie from other movies
    movies_list = sorted(list(enumerate(distances)), reverse = True, key = lambda x:x[1])[1:6]
    for i in movies_list:
        print(new_df.iloc[i[0]].title)
        

In [40]:
suggest("Pirates of the Caribbean: At World's End")

Pirates of the Caribbean: Dead Man's Chest
Pirates of the Caribbean: The Curse of the Black Pearl
Pirates of the Caribbean: On Stranger Tides
20,000 Leagues Under the Sea
Life of Pi


In [41]:
import pickle

In [42]:
new_df.to_dict()

{'movie_id': {0: 19995,
  1: 285,
  2: 206647,
  3: 49026,
  4: 49529,
  5: 559,
  6: 38757,
  7: 99861,
  8: 767,
  9: 209112,
  11: 10764,
  12: 58,
  13: 57201,
  14: 49521,
  15: 2454,
  16: 24428,
  17: 1865,
  18: 41154,
  19: 122917,
  20: 1930,
  21: 20662,
  22: 57158,
  23: 2268,
  24: 254,
  25: 597,
  26: 271110,
  27: 44833,
  28: 135397,
  29: 37724,
  30: 558,
  31: 68721,
  32: 12155,
  33: 36668,
  34: 62211,
  35: 8373,
  36: 91314,
  37: 68728,
  38: 102382,
  39: 20526,
  40: 49013,
  41: 44912,
  42: 10193,
  43: 534,
  44: 168259,
  45: 72190,
  46: 127585,
  47: 54138,
  48: 81005,
  49: 64682,
  50: 9543,
  51: 68726,
  52: 38356,
  53: 217,
  54: 105864,
  55: 62177,
  57: 10681,
  58: 5174,
  59: 14161,
  60: 17979,
  61: 76757,
  62: 258489,
  63: 411,
  64: 246655,
  65: 155,
  67: 15512,
  68: 1726,
  69: 44826,
  70: 8487,
  71: 1735,
  72: 297761,
  73: 2698,
  74: 137113,
  75: 9804,
  76: 14869,
  77: 150540,
  79: 10138,
  80: 58595,
  81: 102651,
  82

In [43]:
pickle.dump(new_df.to_dict(), open('movie_list.pkl','wb'))

In [44]:
pickle.dump(similarity, open('similarity.pkl','wb'))